# Customer Behavior & Revenue Analysis for an Online Retail Store

## Project Overview
This project analyzes transactional data from an online retail store to understand sales performance, customer purchasing behavior, and revenue distribution. Using Python for data analysis and Tableau for visualization, the project focuses on identifying revenue trends, high-value customers, and opportunities for business growth through data-driven insights.

The analysis covers sales trends over time, product performance, customer segmentation, and geographic distribution of revenue.

---

## Business Objective
The primary objective of this project is to:
- Analyze customer purchasing behavior and revenue patterns
- Identify high-value and repeat customers
- Understand sales trends across time and geography
- Provide actionable insights to support customer retention, revenue growth, and strategic decision-making

---

## Dataset Description
The dataset contains transactional data from an online retail store, where each row represents a product-level transaction within an invoice. The data includes information on orders, products, customers, pricing, quantities, and geographic location.

### Key Columns
- **Invoice**: Unique identifier for each transaction (order)
- **StockCode**: Unique product identifier
- **Description**: Product description
- **Quantity**: Number of units purchased (negative values indicate returns)
- **InvoiceDate**: Date and time of the transaction
- **Price**: Unit price of the product
- **Customer ID**: Unique identifier for customers
- **Country**: Customer’s country

---

## Tools Used
- **Python** (Pandas, NumPy) – Data cleaning, transformation, and analysis  
- **Jupyter Notebook** – Analysis and documentation  
- **Tableau** – Interactive dashboards and visual storytelling  
- **GitHub** – Version control and portfolio hosting


In [1]:
#Importing Necessary Libraries
import pandas as pd
import numpy as np

In [3]:
#Load the Dataset
df = pd.read_csv('online_retail_II.csv')

In [5]:
#Checking the head of the dataset
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


Data Cleaning & Preparation

In [6]:
#Checking the rows & columns
df.shape

(1067371, 8)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


In [9]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

In [10]:
# Create a mapping of StockCode to the most frequent Description
description_map = (
    df.dropna(subset=['Description'])
      .groupby('StockCode')['Description']
      .agg(lambda x: x.mode().iloc[0])
)

# Fill missing Description using the mapping
df['Description'] = df['Description'].fillna(df['StockCode'].map(description_map))

# For any remaining missing values (rare cases)
df['Description'] = df['Description'].fillna('Unknown Product')

In [11]:
df_customers = df.dropna(subset=['Customer ID'])

In [12]:
#Filling missing values for Customer ID
df['Customer ID'] = df['Customer ID'].fillna('Guest')

In [13]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

Missing product descriptions were imputed using StockCode-based mapping.

Missing customer IDs represent guest customers and were excluded from customer-level analysis.

In [14]:
#Renaming the columns for easier understandability
df.rename(columns= {'Invoice':'Order_ID', 
                    'StockCode':'Product_Code', 
                    'Description':'Product_Name', 
                    'InvoiceDate':'Order_Datetime', 
                    'Price':'Unit_Price',
                    'Customer ID':'Customer_ID'}, inplace=True)

In [15]:
df.head()

,Order_ID,Product_Code,Product_Name,Quantity,Order_Datetime,Unit_Price,Customer_ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


Removing Cancelled Order_ID and returned items (negative quantities) were excluded to focus the analysis on completed sales transactions

In [17]:
#Remove cancelled invoices
df = df[~df['Order_ID'].astype(str).str.startswith('C')]

In [18]:
#Remove rows with negative quantities (returns)
df = df[df['Quantity'] > 0]

In [19]:
#Creating a new column (revenue) to get the actual price for each row not just the unit price
df['Revenue'] = df['Quantity'] * df['Unit_Price']

In [20]:
#Checking the data types of the columns in the dataset
df.dtypes

Order_ID           object
Product_Code       object
Product_Name       object
Quantity            int64
Order_Datetime     object
Unit_Price        float64
Customer_ID        object
Country            object
Revenue           float64
dtype: object

In [22]:
#Changing the datatype of the Order_datetime from object to Datetime type
df['Order_Datetime'] = pd.to_datetime(df['Order_Datetime'])

In [29]:
df.dtypes

Order_ID                   object
Product_Code               object
Product_Name               object
Quantity                    int64
Order_Datetime     datetime64[ns]
Unit_Price                float64
Customer_ID                object
Country                    object
Revenue                   float64
Order_Date                 object
Order_Monthname            object
dtype: object

In [30]:
#Creating a new column (order_date) to get just the date (DD/MM/YYYY) without the time
df['Order_Date'] = df['Order_Datetime'].dt.strftime('%d-%m-%Y')

In [31]:
#Creating a new column (Order_month) to get just the month
df['Order_Monthname'] = df['Order_Datetime'].dt.month_name()

In [41]:
#Creating a new column (Order_hour) to get just the hour
df['Order_Hour'] = df['Order_Datetime'].dt.strftime('%I %p')

In [37]:
df.drop(columns='Order_Hour_AMPM', axis=1, inplace=True)

In [42]:
df.head()

,Order_ID,Product_Code,Product_Name,Quantity,Order_Datetime,Unit_Price,Customer_ID,Country,Revenue,Order_Date,Order_Monthname,Order_Hour
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4,01-12-2009,December,07 AM
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,01-12-2009,December,07 AM
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,01-12-2009,December,07 AM
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8,01-12-2009,December,07 AM
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0,01-12-2009,December,07 AM


In [40]:
df.dtypes

Order_ID                   object
Product_Code               object
Product_Name               object
Quantity                    int64
Order_Datetime     datetime64[ns]
Unit_Price                float64
Customer_ID                object
Country                    object
Revenue                   float64
Order_Date                 object
Order_Monthname            object
Order_Hour                 object
dtype: object

Sales & Revenue Analysis

In [46]:
#Total revenue
Total_revenue = df['Revenue'].sum() 
Total_revenue

np.float64(20813918.428)

In [50]:
#Monthly revenue trends
monthly_revenue = (
    df.groupby('Order_Monthname')['Revenue'].sum()
)

monthly_revenue

Order_Monthname
April        1165743.253
August       1434289.170
December     2727095.230
February     1076971.626
January      1344073.062
July         1325902.341
June         1514010.040
March        1551209.491
May          1430394.880
November     2979768.812
October      2281537.340
September    1982923.183
Name: Revenue, dtype: float64

In [52]:
#Top 3 best sales months
monthly_revenue.sort_values(ascending=False).head(3)

Order_Monthname
November    2979768.812
December    2727095.230
October     2281537.340
Name: Revenue, dtype: float64

In [53]:
#Top 3 worst sales months
monthly_revenue.sort_values(ascending=False).tail(3)

Order_Monthname
July        1325902.341
April       1165743.253
February    1076971.626
Name: Revenue, dtype: float64

In [55]:
#Revenue by Country
Country_revenue = df.groupby('Country')['Revenue'].sum()

Country_revenue.sort_values(ascending=False)

Country
United Kingdom          1.771230e+07
EIRE                    6.644318e+05
Netherlands             5.542323e+05
Germany                 4.312625e+05
France                  3.569446e+05
Australia               1.699681e+05
Spain                   1.091785e+05
Switzerland             1.010113e+05
Sweden                  9.190372e+04
Denmark                 6.986219e+04
Belgium                 6.575342e+04
Norway                  6.010962e+04
Portugal                5.801665e+04
Japan                   4.713839e+04
Channel Islands         4.499676e+04
Italy                   3.255042e+04
Finland                 2.992554e+04
Singapore               2.531706e+04
Cyprus                  2.498013e+04
Hong Kong               2.389432e+04
Austria                 2.361301e+04
Greece                  1.909619e+04
Israel                  1.133466e+04
United Arab Emirates    1.109763e+04
Unspecified             1.093601e+04
Poland                  1.090245e+04
USA                     8.3668

In [60]:
#Top 10 products by revenue
Product_Revenue = df.groupby('Product_Name')['Revenue'].sum()

Product_Revenue.sort_values(ascending=False).head(10)

Product_Name
REGENCY CAKESTAND 3 TIER              344563.25
Manual                                340731.33
DOTCOM POSTAGE                        322657.48
WHITE HANGING HEART T-LIGHT HOLDER    266923.55
PAPER CRAFT , LITTLE BIRDIE           168469.60
JUMBO BAG RED RETROSPOT               150935.56
PARTY BUNTING                         149187.05
ASSORTED COLOUR BIRD ORNAMENT         132187.92
POSTAGE                               127597.42
PAPER CHAIN KIT 50'S CHRISTMAS        123141.54
Name: Revenue, dtype: float64

Customer Analysis

In [61]:
#Number of unique customers
df['Customer_ID'].nunique()

5882

In [65]:
#Revenue per customer
Revenue_per_customer = df.groupby('Customer_ID')['Revenue'].sum()

In [66]:
#Top 10 customers by revenue
Revenue_per_customer.sort_values(ascending=False).head(10)

Customer_ID
Guest      3070489.25
18102.0     608821.65
14646.0     528602.52
14156.0     313946.37
14911.0     295972.63
17450.0     246973.09
13694.0     196482.81
17511.0     175603.55
16446.0     168472.50
16684.0     147142.77
Name: Revenue, dtype: float64

In [70]:
#Getting Repeat vs One-Time Buyer
Customer_orders = df.groupby('Customer_ID')['Order_ID'].nunique()

Customer_orders

Customer_ID
12346.0      12
12347.0       8
12348.0       5
12349.0       4
12350.0       1
           ... 
18284.0       1
18285.0       1
18286.0       2
18287.0       7
Guest      4968
Name: Order_ID, Length: 5882, dtype: int64

In [73]:
buyer_type = Customer_orders.apply(
    lambda x: 'One-time Buyer' if x == 1 else 'Repeat Buyer'
)

buyer_type.value_counts()

Order_ID
Repeat Buyer      4256
One-time Buyer    1626
Name: count, dtype: int64

RFM ANALYSIS

RECENCY (R)

In [74]:
#measures the number of days since a customer’s most recent purchase
reference_date = df['Order_Datetime'].max()

recency = (
    df.groupby('Customer_ID')['Order_Datetime']
      .max()
      .apply(lambda x: (reference_date - x).days)
)

recency.sort_values(ascending=False).head()

Customer_ID
17592.0    738
12636.0    738
13526.0    737
17087.0    737
17056.0    737
Name: Order_Datetime, dtype: int64

In [75]:
buyer_type = Customer_orders.apply(
    lambda x: 'One-time Buyer' if x == 1 else 'Repeat Buyer'
)

In [76]:
#To check active and inactive customers
recent_customers = recency.apply(
    lambda x: 'Active Customers' if x <= 30 else 'Inactive Customers'
)

recent_customers.value_counts()

Order_Datetime
Inactive Customers    4186
Active Customers      1696
Name: count, dtype: int64

Frequency (F)

In [77]:
#Checking distinct order a customer has placed
frequency = df.groupby('Customer_ID')['Order_ID'].nunique()

frequency.sort_values(ascending= False).head()

Customer_ID
Guest      4968
14911.0     398
12748.0     337
17841.0     211
15311.0     208
Name: Order_ID, dtype: int64

In [78]:
#Checking for frequent and Occasional buyers
High_end_customer = frequency.apply(
    lambda x: 'Frequent customer' if x  >= 50 else 'Occasional buyer'
      )

High_end_customer.value_counts()

Order_ID
Occasional buyer     5827
Frequent customer      55
Name: count, dtype: int64

Monetary (M)

In [79]:
#Checking for revenue generated by customer
monetary = df.groupby('Customer_ID')['Revenue'].sum()

monetary.sort_values(ascending= False).head()

Customer_ID
Guest      3070489.25
18102.0     608821.65
14646.0     528602.52
14156.0     313946.37
14911.0     295972.63
Name: Revenue, dtype: float64

In [80]:
#Checking for High Spending Customers
High_spender = monetary.apply(
    lambda x: 'High value customers' if x >= 100000 else 'Low spend customer'
)

High_spender.value_counts()

Revenue
Low spend customer      5866
High value customers      16
Name: count, dtype: int64

In [81]:
#Combining R,F,M into one table
RFM = pd.DataFrame({
    'Recency': recency,
    'Frequency': frequency,
    'Monetary': monetary
})

RFM

,Recency,Frequency,Monetary
Customer_ID,,,
12346.0,325,12,77556.46
12347.0,1,8,5633.32
12348.0,74,5,2019.40
12349.0,18,4,4428.69
12350.0,309,1,334.40
...,...,...,...
18284.0,431,1,461.68
18285.0,660,1,427.00
18286.0,476,2,1296.43


In [82]:
# Divide customer metrics into four ranked quartiles for scoring.
RFM['R_Score'] = pd.qcut(RFM['Recency'].rank(method='first', ascending=False), 4, labels=[4,3,2,1])
RFM['F_Score'] = pd.qcut(RFM['Frequency'].rank(method='first'), 4, labels=[1,2,3,4])
RFM['M_Score'] = pd.qcut(RFM['Monetary'].rank(method='first'), 4, labels=[1,2,3,4])

In [85]:
#Combining scores into a 3-digit string to identify customer segments
RFM['RFM_Score'] = (
    RFM['R_Score'].astype(str) +
    RFM['F_Score'].astype(str) +
    RFM['M_Score'].astype(str)
)

RFM['RFM_Score']

Customer_ID
12346.0    344
12347.0    144
12348.0    233
12349.0    134
12350.0    311
          ... 
18284.0    422
18285.0    422
18286.0    423
18287.0    244
Guest      144
Name: RFM_Score, Length: 5882, dtype: object

In [84]:
def segment_customer(row):
    if row['RFM_Score'] == 444:
        return 'Champions'
    elif row['F_Score'] >= 3 and row['M_Score'] >= 3:
        return 'Loyal Customers'
    elif row['R_Score'] == 1:
        return 'At Risk'
    else:
        return 'Others'

RFM['Segment'] = RFM.apply(segment_customer, axis=1)

In [91]:
RFM.head()

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Segment
0,325,12,77556.46,3,4,4,344,Loyal Customers
1,1,8,5633.32,1,4,4,144,Loyal Customers
2,74,5,2019.40,2,3,3,233,Loyal Customers
3,18,4,4428.69,1,3,4,134,Loyal Customers
4,309,1,334.40,3,1,1,311,Others


In [92]:
RFM['Customer_ID'] = df['Customer_ID']

In [93]:
RFM.head()

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,Segment,Customer_ID
0,325,12,77556.46,3,4,4,344,Loyal Customers,13085.0
1,1,8,5633.32,1,4,4,144,Loyal Customers,13085.0
2,74,5,2019.40,2,3,3,233,Loyal Customers,13085.0
3,18,4,4428.69,1,3,4,134,Loyal Customers,13085.0
4,309,1,334.40,3,1,1,311,Others,13085.0


In [87]:
#Customers per segment count
RFM['Segment'].value_counts()

Segment
Others             2970
Loyal Customers    2465
At Risk             447
Name: count, dtype: int64

In [88]:
#Revenue by segment
RFM.groupby('Segment')['Monetary'].sum().sort_values(ascending=False)

Segment
Loyal Customers    1.866239e+07
Others             1.693876e+06
At Risk            4.576485e+05
Name: Monetary, dtype: float64

“RFM analysis was used to segment customers based on purchasing behavior, enabling identification of high-value, loyal, and at-risk customers.”

In [94]:
df.to_csv('renewed.csv', index=False)

In [96]:
RFM.to_csv('renewed_sec.csv', index=False)